# Réseau de Neurones Convolutionnels

## Importation des librairies

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [3]:
IMG_SIZE = 150
NUM_CHANNELS = 3
TARGET_SIZE = (IMG_SIZE, IMG_SIZE)
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, NUM_CHANNELS)
BATCH_SIZE = 16

DATA_ROOT = "./dataset"

## Préparation des images

In [4]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train = datagen.flow_from_directory(DATA_ROOT,
                                    target_size=(IMG_SIZE, IMG_SIZE),
                                    batch_size=BATCH_SIZE,
                                    class_mode='categorical',
                                    subset='training')
val = datagen.flow_from_directory(DATA_ROOT,
                                  target_size=TARGET_SIZE,
                                  batch_size=BATCH_SIZE,
                                  class_mode='categorical',
                                  subset='validation')

Found 442 images belonging to 5 classes.
Found 109 images belonging to 5 classes.


In [5]:
num_classes = len(train.class_indices)
num_classes

5

## Architecture du modèle

In [6]:
model = Sequential()

model.add(Conv2D(filters=128, kernel_size=3, activation='relu', input_shape=(INPUT_SHAPE)))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Flatten())

model.add(Dense(units=512, activation='relu'))

model.add(Dense(units=num_classes, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        36896     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       36992     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

## Optimisation

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

## Entraînement

In [ ]:
EPOCHS = 10

steps_per_epoch = train.samples // BATCH_SIZE

history = model.fit(train,
                    epochs=EPOCHS,
                    steps_per_epoch=steps_per_epoch)

Epoch 1/10
 3/27 [==>...........................] - ETA: 1:29 - loss: 2.4989 - acc: 0.2500

/home/joseph/anaconda3/envs/tf.2.0/lib/python3.7/site-packages/PIL/Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


26/27 [===========================>..] - ETA: 5s - loss: 1.7139 - acc: 0.2692 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
loss = history.history['loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(8, 4))
plt.plot(epochs, acc, label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(epochs, loss, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.show()

## Évaluation

In [ ]:
steps_per_epoch = val.samples // BATCH_SIZE

res = model.evaluate(val,
                     steps=steps_per_epoch)
print(f'Loss : {res[0]} - Accuracy: {res[1]}')

## Sauvegarde

In [ ]:
model.save('model.h5')